In [1]:
import subprocess
import tempfile
import os
import pandas as pd
import time
import glob

In [15]:
def run_cpp_code_with_file_input(code: str, input_file_path: str) -> str:
    # Create a temporary directory to hold the C++ file and executable
    with tempfile.TemporaryDirectory() as temp_dir:
        cpp_file_path = os.path.join(temp_dir, "code.cpp")
        executable_path = os.path.join(temp_dir, "code")
        
        # Write the C++ code to a file
        with open(cpp_file_path, "w") as cpp_file:
            cpp_file.write(code)
        
        # Compile the C++ code
        compile_process = subprocess.run(["g++", cpp_file_path, "-o", executable_path], capture_output=True, text=True)
        if compile_process.returncode != 0:
            # Compilation failed
            return f"Compilation Failed:\n{compile_process.stderr}"
        
        # Run the compiled executable with input redirected from the input file
        try:
            start_time = time.time()
            with open(input_file_path, 'r') as input_file:
                run_process = subprocess.run(executable_path, stdin=input_file, capture_output=True, text=True, universal_newlines=True, timeout=2)
                if run_process.returncode != 0:
                    # Runtime error
                    return f"Runtime Error:\n{run_process.stderr}"
            end_time = time.time()
            return run_process.stdout, (end_time - start_time)
        except subprocess.TimeoutExpired:
            return -1, -1

def eval_output(output: str, expected_output_file: str) -> bool:
    with open(expected_output_file, 'r') as expected_file:
        expected_output = expected_file.read()
        return output.strip() == expected_output.strip()
    

def run_tcs(code: str, sample_output_folder: str, hidden_output_folder) -> bool:
    for folder in [sample_output_folder, hidden_output_folder]:
        input_files = glob.glob(os.path.join(folder, "input.*.txt"))
        for input_file in input_files:
            # print('processing:', input_file)
            expected_output_file = input_file.replace('input', 'output')  # Assuming expected output file naming convention
            actual_output, time = run_cpp_code_with_file_input(code, input_file)
            # print(time)
            if actual_output is None or not eval_output(actual_output, expected_output_file):
                print('Failed on test case', input_file)
                return False
    
    return True

In [3]:
df = pd.read_csv("../datasets/improvement_pairs_additional_metadata.csv", sep="\t")

In [16]:
sample_code = df.at[4, 'code_v1']

In [17]:
print(run_tcs(sample_code, "../datasets/codenet/public_test_cases/p00849/", "../datasets/codenet2/generated_test_cases/p00849/"))

True


In [6]:
df.head()

,user_id,problem_id,language,submission_id_v0,submission_id_v1,cpu_time_v0,cpu_time_v1,memory_v0,memory_v1,status_v0,status_v1,improvement_frac,code_v0,code_v1,code_v0_loc,code_v1_loc
0,u116766943,p00849,C++,s624998539,s546809015,9370.0,9250.0,1232.0,1228.0,Time Limit Exceeded,Time Limit Exceeded,1.28,"#include<bits/stdc++.h>\n\n\n\n#define REP(i,s...","#include<bits/stdc++.h>\n\n\n\n#define REP(i,s...",122.0,127.0
1,u116766943,p00849,C++,s546809015,s829771916,9250.0,8780.0,1228.0,1228.0,Time Limit Exceeded,Time Limit Exceeded,5.08,"#include<bits/stdc++.h>\n\n\n\n#define REP(i,s...","#include<bits/stdc++.h>\n\n\n\n#define REP(i,s...",127.0,125.0
2,u116766943,p00849,C++,s812631681,s154463767,8700.0,8030.0,1232.0,1232.0,Time Limit Exceeded,Time Limit Exceeded,7.70,"#include<bits/stdc++.h>\n\n\n\n#define REP(i,s...","#include<bits/stdc++.h>\n\n\n\n#define REP(i,s...",144.0,144.0
3,u116766943,p00849,C++,s363969251,s061168313,8500.0,920.0,1252.0,12164.0,Time Limit Exceeded,Accepted,89.18,"#include<bits/stdc++.h>\n\n\n\n#define REP(i,s...","#include<bits/stdc++.h>\n\n\n\n#define REP(i,s...",145.0,214.0
4,u116766943,p00849,C++,s061168313,s755775167,920.0,420.0,12164.0,9780.0,Accepted,Accepted,54.35,"#include<bits/stdc++.h>\n\n\n\n#define REP(i,s...","#include<bits/stdc++.h>\n\n\n\n#define REP(i,s...",214.0,178.0
